<a href="https://colab.research.google.com/github/mvdheram/Stereotypical-Social-bias-detection-/blob/Machine-learning-classifiers/Feature_based_machine_learning_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Features based machine learning models 

* Reference : 
    1. Linguistic models for detecting bias https://aclanthology.org/P13-1162.pdf
    2. Automatically Neutralizing Subjective Bias in Text https://ojs.aaai.org/index.php/AAAI/article/view/5385 

Features :

* Bias lexicons with count
* Sentiment 
* Generic words NNS and NNPS
* Toxicity 
* Generic features 
* A, AE names

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Trained models/mult_label_dataset/ohe_multilabel.csv', index_col = 0)

In [ ]:
df

## Pre-processing

In [ ]:
# Tokenization using spacy
import spacy

nlp = spacy.load("en_core_web_sm")

def tokenize(text):  

  doc = nlp(text)
  tokens = [token.text.lower() for token in doc]
  return tokens

def lemmatization(text):

  doc = nlp(text)
  lemmas = [token.lemma_.lower() for token in doc]
  return lemmas

# Remove tokens that are not alphabetic - depends on particular application 
def clean_text(text):

  lemmas = lemmatization(text)
  a_lemmas = [lemma for  lemma in lemmas
              if lemma.isalpha()]
  
  return (' '.join(a_lemmas))


# Remove stopwords - Update according to stereotypical bias 
def remove_stopwords(text):

  stopwords = spacy.lang.en.stop_words.STOP_WORDS
  
  lemmas = lemmatization(text)
  a_lemmas = [lemma for  lemma in lemmas
              if lemma.isalpha() and lemma not in stopwords]
  
  return (' '.join(a_lemmas))


# Parts of speech tagger 
def pos_tags(text):
  
  doc = nlp(text)
  pos = [(token.text, token.tag_) for token in doc]
  return (pos)


# Named entity recognition 
def ner_tags(text):
  
  doc = nlp(text)
  ner = [(ent.text, ent.label_) for ent in doc.ents]
  return (ner)

In [ ]:
stereo = df.copy()
stereo['clean_text'] = stereo['sentence'].apply(clean_text)

In [ ]:
stereo['pos_tags'] = stereo['clean_text'].apply(pos_tags) 

In [ ]:
stereo['lemmatized_withStopwords']= stereo['clean_text'].apply(remove_stopwords)

In [ ]:
stereo['tokenized_lemmatized_withstopwords'] = stereo['lemmatized_withStopwords'].apply(tokenize)

In [ ]:
stereo['lemma_pos'] = stereo['lemmatized_withStopwords'].apply(pos_tags)

In [ ]:
stereo['Ner_tags'] = stereo['clean_text'].apply(ner_tags)

In [ ]:
stereo

,Ethnicity,gender,profession,religion,Anti-stereotype,stereotype,unrelated,sentence,clean_text,lemmatized_withStopwords,tokenized_lemmatized_withstopwords
0,1,0,0,0,0,1,0,Many people live in Ethiopia. The people are v...,many people live in ethiopia the people be ver...,people live ethiopia people thin good distance...,"[people, live, ethiopia, people, thin, good, d..."
1,1,0,0,0,0,1,0,My professor is a hispanic man He came here il...,professor be a hispanic man come here illegally,professor hispanic man come illegally,"[professor, hispanic, man, come, illegally]"
2,0,1,0,0,0,1,0,The schoolgirl is walking down the street. She...,the schoolgirl be walk down the street be a ta...,schoolgirl walk street target child predator,"[schoolgirl, walk, street, target, child, pred..."
3,1,0,0,0,0,1,0,We'll likely fly over Cape Verde on our flight...,will likely fly over cape verde on flight tomo...,likely fly cape verde flight tomorrow bet peop...,"[likely, fly, cape, verde, flight, tomorrow, b..."
4,1,0,0,0,0,1,0,I know many people who are Russian. They love ...,know many people who be russian love vodka,know people russian love vodka,"[know, people, russian, love, vodka]"
...,...,...,...,...,...,...,...,...,...,...,...
16539,0,0,0,0,0,0,1,Arab people are associated with hi.,arab people be associate with hi,arab people associate hi,"[arab, people, associate, hi]"
16540,0,0,0,0,0,0,1,Car was a new chemist in our lab.,car be a new chemist in lab,car new chemist lab,"[car, new, chemist, lab]"
16541,0,0,0,0,0,0,1,"The gentlemen stood around the fire, carefully...",the gentleman stand around the fire carefully ...,gentleman stand fire carefully pass boxing,"[gentleman, stand, fire, carefully, pass, boxing]"
16542,0,0,0,0,0,0,1,The common ukrainian is a supporter of a floun...,the common ukrainian be a supporter of a floun...,common ukrainian supporter flounder run govern...,"[common, ukrainian, supporter, flounder, run, ..."


In [ ]:
stereo.to_csv('stereo_features.csv')

## Feature engineering

Scoring features :


* Readability tests :
  https://pypi.org/project/textatistic/
  * Determine readability of english passage
  * Scale ranging from primary school up to college graduate level
  * A mathematical formula utilizing word, syllabel and sentence count
  * Used in fake news and opinion spam detection 

  Types :

  1. Flesch reading ease : 

    * The higher the score, the better the readability. 
    * score of 0-30 implies only college graduates can understand while 90-100 implies that a 5th grade student can understand.
    
    Two factors :

      1. Greater the average sentence length, harder the text to read
      2. Greater the average number of syllables, harder the text to read

* Avg_tf_idf
* Max_tf_idf
* Number of characters 
* Word count
* Average word length
* Vadar Sentiment analysis
* Text subjectivity (Text blob)
* Toxicity analysis (detoxify)



In [ ]:
import pandas as pd

stereo = pd.read_csv('/content/drive/MyDrive/Trained models/mult_label_dataset/stereo_features_f.csv',index_col=0)

In [ ]:
stereo

,Ethnicity,gender,profession,religion,Anti-stereotype,stereotype,unrelated,sentence,clean_text,pos,lemmatized_withStopwords,lemma_pos,Ner_tags
0,1,0,0,0,0,1,0,Many people live in Ethiopia. The people are v...,many people live in ethiopia the people be ver...,"[('many', 'JJ'), ('people', 'NNS'), ('live', '...",people live ethiopia people thin good distance...,"[('people', 'NNS'), ('live', 'VBP'), ('ethiopi...","[('ethiopia', 'GPE')]"
1,1,0,0,0,0,1,0,My professor is a hispanic man He came here il...,professor be a hispanic man come here illegally,"[('professor', 'NNP'), ('be', 'VB'), ('a', 'DT...",professor hispanic man come illegally,"[('professor', 'NNP'), ('hispanic', 'NNP'), ('...","[('hispanic', 'NORP')]"
2,0,1,0,0,0,1,0,The schoolgirl is walking down the street. She...,the schoolgirl be walk down the street be a ta...,"[('the', 'DT'), ('schoolgirl', 'NN'), ('be', '...",schoolgirl walk street target child predator,"[('schoolgirl', 'NNP'), ('walk', 'NNP'), ('str...",[]
3,1,0,0,0,0,1,0,We'll likely fly over Cape Verde on our flight...,will likely fly over cape verde on flight tomo...,"[('will', 'MD'), ('likely', 'RB'), ('fly', 'VB...",likely fly cape verde flight tomorrow bet peop...,"[('likely', 'RB'), ('fly', 'VB'), ('cape', 'NN...","[('tomorrow', 'DATE')]"
4,1,0,0,0,0,1,0,I know many people who are Russian. They love ...,know many people who be russian love vodka,"[('know', 'VBP'), ('many', 'JJ'), ('people', '...",know people russian love vodka,"[('know', 'VBP'), ('people', 'NNS'), ('russian...","[('russian', 'NORP')]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16539,0,0,0,0,0,0,1,Arab people are associated with hi.,arab people be associate with hi,"[('arab', 'JJ'), ('people', 'NNS'), ('be', 'VB...",arab people associate hi,"[('arab', 'JJ'), ('people', 'NNS'), ('associat...","[('arab', 'NORP')]"
16540,0,0,0,0,0,0,1,Car was a new chemist in our lab.,car be a new chemist in lab,"[('car', 'NN'), ('be', 'VB'), ('a', 'DT'), ('n...",car new chemist lab,"[('car', 'NN'), ('new', 'JJ'), ('chemist', 'NN...",[]
16541,0,0,0,0,0,0,1,"The gentlemen stood around the fire, carefully...",the gentleman stand around the fire carefully ...,"[('the', 'DT'), ('gentleman', 'NNP'), ('stand'...",gentleman stand fire carefully pass boxing,"[('gentleman', 'NNP'), ('stand', 'VB'), ('fire...",[]
16542,0,0,0,0,0,0,1,The common ukrainian is a supporter of a floun...,the common ukrainian be a supporter of a floun...,"[('the', 'DT'), ('common', 'JJ'), ('ukrainian'...",common ukrainian supporter flounder run govern...,"[('common', 'JJ'), ('ukrainian', 'JJ'), ('supp...","[('ukrainian', 'NORP')]"


In [ ]:
scoring_features = stereo.copy()

In [ ]:
scoring_features.drop(['pos','lemma_pos',	'Ner_tags'],axis=1, inplace= True)

In [ ]:
 # Number of characters
 scoring_features['num_chars']  = scoring_features['sentence'].apply(len)

In [ ]:
# Number of words
def word_count(string):
  # split the string into words
  words = string.split()

  # Return length of words list
  return len(words)

scoring_features['num_words'] = scoring_features['sentence'].apply(word_count)

In [ ]:
# Average word length
def avg_word_length(x):

  # Split the string into words
  words = x.split()

  # Compute length of each word and store in a seperate list
  word_lengths = [len(word) for word in words]

  # Compute average word length 
  try:
    avg_word_length = sum(word_lengths)/len(words)
  except ZeroDivisionError:
    avg_word_length = 0

  return (avg_word_length)

scoring_features['avg_word_length'] = scoring_features['sentence'].apply(avg_word_length) 

In [ ]:
scoring_features.columns

Index(['Ethnicity', 'gender', 'profession', 'religion', 'Anti-stereotype',
       'stereotype', 'unrelated', 'sentence', 'clean_text',
       'lemmatized_withStopwords', 'num_chars', 'num_words',
       'avg_word_length'],
      dtype='object')

In [ ]:
pip install textstat

In [ ]:
# Readability tests using textatistic library 
# Import the textatistic class
import textstat
import math

def readability_scores(text):
  # if text.endswith(".") == False:
  #   text = text+"."
  readability_score = textstat.flesch_reading_ease(text)

  # Generate scores
  return readability_score

In [ ]:
try:
  scoring_features['flesch_score'] = scoring_features['sentence'].apply(readability_scores)
except ZeroDivisionError:
  scoring_features['flesch_score'] = 0

In [ ]:
pip install -U textblob

In [ ]:
from textblob import TextBlob

def get_subjectivity(text):
    try:
        textblob = TextBlob(unicode(text, 'utf-8'))
        subj = textblob.sentiment.subjectivity
    except:
        subj = 0.0
    return subj

In [ ]:
scoring_features['subjectivity_score'] = scoring_features['sentence'].apply(get_subjectivity)

Vectorization :

* n_grams
* tf_idf 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Building n-gram models - capture context
# Range = (2,2) - bi-grams, (1,3) - unigram, bigram, trigram
def n_grams(range, corpus):
  # Bag of words feature - docxterm matrix 
  vectorizer = CountVectorizer(ngram_range = range)
  corpus = corpus.values.astype('U')
  bow_matrix = vectorizer.fit_transform(corpus)
  cv_df = pd.DataFrame(bow_matrix.toarray(), columns = vectorizer.get_feature_names()).add_prefix('Counts_')
  # corpus = pd.concat([corpus,cv_df],axis = 1, sort = False)
  return cv_df

In [ ]:
# tf-idf  - higher the weight more the importance 
# Used for train set
from sklearn.feature_extraction.text import TfidfVectorizer

def tf_idf(corpus):
  vectorizer = TfidfVectorizer()
  vectorizer = TfidfVectorizer(max_features = 10000)
  corpus = corpus.values.astype('U')
  tfidf_matrix = vectorizer.fit_transform(corpus)
  tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns = vectorizer.get_feature_names()).add_prefix('tfIdf_')
  # corpus = pd.concat([corpus,tfidf_df],axis = 1, sort = False)
  return tfidf_df

In [ ]:
df.columns

Index(['Ethnicity', 'gender', 'profession', 'religion', 'Anti-stereotype',
       'stereotype', 'unrelated', 'sentence', 'clean_text', 'pos_tags',
       'lemmatized_withStopwords', 'tokenized_lemmatized_withstopwords',
       'lemma_pos', 'Ner_tags'],
      dtype='object')

In [ ]:
tf_idf_feature = tf_idf(df['tokenized_lemmatized_withstopwords'])

In [ ]:
# Inspect the different words being values after BOW and tfidf transformation 
def examine_row(corpus,row_n):
  examine_row = corpus.iloc[row_n]
  print(examine_row.sort_values(ascending= False).head())
  total = corpus.sum()
  print("Total sum of the counts per word \n",total.head()) # Total sum of the counts per word
  # print("Sums sorted: ",total.sort_values(ascending= False).head())

In [ ]:
clean_text = scoring_features['clean_text']
tfidf = TfidfVectorizer()
# corpus = clean_text.values.astype('U')
# Whole dataset has to  be given for tfidf model
tfidf_model = tfidf.fit(corpus)

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import numpy as np
import nltk

def tfidf_max_avg_features(sent):
    avg_tfidf_feature = 0
    max_tfidf_feature = 0
    tokenized_words = nltk.word_tokenize(sent)
    tfidf_vector = tfidf_model.transform([sent])
    if avg == True:
      avg_tfidf_feature = np.sum(tfidf_vector.toarray())/len(tokenized_words)
      return avg_tfidf_feature
    else:
      max_tfidf_feature = np.max(tfidf_vector.toarray())
      return max_tfidf_feature

In [ ]:
avg = True
scoring_features['avg_tfidf_feature'] = scoring_features['sentence'].apply(tfidf_max_avg_features)

In [ ]:
avg = False
scoring_features['max_tfidf_feature'] = scoring_features['sentence'].apply(tfidf_max_avg_features)

Vadar sentiment analysis

In [ ]:
pip install vaderSentiment

     |████████████████████████████████| 125 kB 6.7 MB/s 


In [ ]:
# Sentiment analysis 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

def vader_sentiment(text):
  score = analyser.polarity_scores(text)
  return score

senti = scoring_features['sentence'].apply(vader_sentiment) 
scoring_features = pd.concat([scoring_features,(pd.DataFrame.from_dict(dict(senti).values()))],axis = 1, sort = False)
# scoring_features.head()

In [ ]:
pip install detoxify

     |████████████████████████████████| 2.6 MB 7.4 MB/s 
     |████████████████████████████████| 1.2 MB 52.9 MB/s 
     |████████████████████████████████| 895 kB 37.4 MB/s 
     |████████████████████████████████| 636 kB 71.6 MB/s 
     |████████████████████████████████| 3.3 MB 62.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# Toxicity identification 
from detoxify import Detoxify

def toxicity(text):
  results = Detoxify('original').predict(text)
  return math.floor(results['toxicity']*100)

scoring_features['toxicity'] = scoring_features['sentence'].apply(toxicity) 

In [ ]:
scoring_features.to_csv('/content/drive/MyDrive/temp_df.csv')

Count based features :

* Lexicons - Counts
  * Hedge in context  - two words around W is a hedge (Hyland, 2005  (e.g., apparently).
  * Factive verb  - w is in Hooper’s (1975) list of factives (e.g., realize).
  * Factive verb in context One/two word(s) around w is a factive (Hooper, 1975)
  * Assertive verb
  * Assertive verb in context 
  * Assertive verb 
  * Implicative verb in context
  * Report verb
  * Entailment (Not found)
  * Entailment in context (Not found)
  * Strong subjective (Used textblob subjectivity score)
  * Weak subjective ((Used textblob subjectivity score)
  * Positive word (Vadar sentiment score)
  * Positive word in context (Vadar sentiment score)
  * Negative word (Vadar sentiment score)
  * Negative word in context (Vadar sentiment score)
  * Grammatical relation - {root,subj,...}
  * Bias lexicon
* Social category target words used in dataset( Characteristic words of each bias type ; e.g. Racial, gender, ..) and scoring_features_pos_Ner
* Characteristic stereotypical words 
* POS :
  * POS(word) : POS of word w 
  * POS(word) - 1 :  POS of one word before w
  * POS(word) + 1  : POS of one word after w


In [ ]:
import pandas as pd

scoring_features = pd.read_csv('/content/drive/MyDrive/temp_df.csv', index_col = 0)

Lexicons

In [ ]:
import json 

f = open('/content/Subjectivity_lexicon.json')

lexicons = json.load(f)

In [ ]:
for keys, value in lexicons.items():
  print(keys,'->',len(value.split('\n')))

assertive_verbs.txt -> 66
bias_lexicon.txt -> 655
bias_word_list_01_2018.txt -> 9742
factive_verbs.txt -> 27
hedges_hyland2005.txt -> 100
implicative_verbs.txt -> 32
report_verbs.txt -> 181
subjectivityClues_lexicon.txt -> 8223


In [ ]:
keys = lexicons.keys()

In [ ]:
def count_lexicon(text):
  count = 0
  try:
    for token in lexicon:
      if token in text:
        count +=1
      else:
        continue
  except :
    pass
  return count

Assertive verbs

In [ ]:
lexicon = set(tokenize(lexicons['assertive_verbs.txt']))

In [ ]:
scoring_features['assertive_verbs_count'] = scoring_features['lemmatized_withStopwords'].apply(count_lexicon)

In [ ]:
len(scoring_features[scoring_features['assertive_verbs_count'] != 0])

1572

Factive verbs

In [ ]:
lexicon = set(tokenize(lexicons['factive_verbs.txt']))

In [ ]:
scoring_features['factive_verbs_count'] = scoring_features['lemmatized_withStopwords'].apply(count_lexicon)

In [ ]:
len(scoring_features[scoring_features['factive_verbs_count'] != 0])

1725

Hedges

In [ ]:
lexicon = set(tokenize(lexicons['hedges_hyland2005.txt']))

In [ ]:
scoring_features['hedges_count'] = scoring_features['lemmatized_withStopwords'].apply(count_lexicon)

In [ ]:
len(scoring_features[scoring_features['hedges_count'] != 0])

9458

Implicative_verbs

In [ ]:
lexicon = set(tokenize(lexicons['implicative_verbs.txt']))

In [ ]:
scoring_features['implicative_verbs_count'] = scoring_features['lemmatized_withStopwords'].apply(count_lexicon)

In [ ]:
len(scoring_features[scoring_features['implicative_verbs_count'] != 0])

1719

Report_verbs

In [ ]:
lexicon = set(tokenize(lexicons['report_verbs.txt']))

In [ ]:
scoring_features['report_verbs_count'] = scoring_features['lemmatized_withStopwords'].apply(count_lexicon)

In [ ]:
len(scoring_features[scoring_features['report_verbs_count'] != 0])

3922

Bias_word_list_01_2018

In [ ]:
lexicon = set(tokenize(lexicons['bias_word_list_01_2018.txt']))

In [ ]:
scoring_features['bias_word_list_01_2018_count'] = scoring_features['lemmatized_withStopwords'].apply(count_lexicon)

In [ ]:
len(scoring_features[scoring_features['bias_word_list_01_2018_count'] != 0])

15198

SubjectivityClues_lexicon

In [ ]:
lexicon = set(tokenize(lexicons['subjectivityClues_lexicon.txt']))

In [ ]:
scoring_features['subjectivityClues_lexicon_count'] = scoring_features['lemmatized_withStopwords'].apply(count_lexicon)

In [ ]:
len(scoring_features[scoring_features['subjectivityClues_lexicon_count'] != 0])

15831

In [ ]:
scoring_features.to_csv("scoring_features.csv")

POS tags Features

In [ ]:
features_pos_ner = pd.read_csv('/content/drive/MyDrive/Trained_models/Results_compilation /Unrelated_adjusted/Baselines/Feature based/Feature_df/stereo_features_pos_ner.csv',index_col = 0)

In [ ]:
features_pos_ner = features_pos_ner.iloc[:,9:]

In [ ]:
features_pos_ner

,pos_tags,lemmatized_withStopwords,tokenized_lemmatized_withstopwords,lemma_pos,Ner_tags
0,"[('many', 'JJ'), ('people', 'NNS'), ('live', '...",people live ethiopia people thin good distance...,"['people', 'live', 'ethiopia', 'people', 'thin...","[('people', 'NNS'), ('live', 'VBP'), ('ethiopi...","[('ethiopia', 'GPE')]"
1,"[('professor', 'NNP'), ('be', 'VB'), ('a', 'DT...",professor hispanic man come illegally,"['professor', 'hispanic', 'man', 'come', 'ille...","[('professor', 'NNP'), ('hispanic', 'NNP'), ('...","[('hispanic', 'NORP')]"
2,"[('the', 'DT'), ('schoolgirl', 'NN'), ('be', '...",schoolgirl walk street target child predator,"['schoolgirl', 'walk', 'street', 'target', 'ch...","[('schoolgirl', 'NNP'), ('walk', 'NNP'), ('str...",[]
3,"[('will', 'MD'), ('likely', 'RB'), ('fly', 'VB...",likely fly cape verde flight tomorrow bet peop...,"['likely', 'fly', 'cape', 'verde', 'flight', '...","[('likely', 'RB'), ('fly', 'VB'), ('cape', 'NN...","[('tomorrow', 'DATE')]"
4,"[('know', 'VBP'), ('many', 'JJ'), ('people', '...",know people russian love vodka,"['know', 'people', 'russian', 'love', 'vodka']","[('know', 'VBP'), ('people', 'NNS'), ('russian...","[('russian', 'NORP')]"
...,...,...,...,...,...
16555,"[('cookie', 'NN'), ('be', 'VB'), ('good', 'JJ'...",cookie good substitute liquid milk,"['cookie', 'good', 'substitute', 'liquid', 'mi...","[('cookie', 'NNP'), ('good', 'JJ'), ('substitu...",[]
16556,"[('jollof', 'NNP'), ('rice', 'NNP'), ('cereal'...",jollof rice cereal totally jambalaya,"['jollof', 'rice', 'cereal', 'totally', 'jamba...","[('jollof', 'NNP'), ('rice', 'NNP'), ('cereal'...","[('jollof rice', 'PERSON')]"
16557,"[('bike', 'NN'), ('out', 'RP'), ('be', 'VB'), ...",bike economical energy efficient mode unk tran...,"['bike', 'economical', 'energy', 'efficient', ...","[('bike', 'NNP'), ('economical', 'JJ'), ('ener...",[]
16558,"[('may', 'MD'), ('see', 'VB'), ('the', 'DT'), ...",little dog need food park grocery store find f...,"['little', 'dog', 'need', 'food', 'park', 'gro...","[('little', 'JJ'), ('dog', 'NN'), ('need', 'VB...",[]


In [ ]:
scoring_features = pd.concat([scoring_features,features_pos_ner],axis =1)

In [ ]:
scoring_features.columns

Index(['Ethnicity', 'gender', 'profession', 'religion', 'Anti-stereotype',
       'stereotype', 'unrelated', 'sentence', 'clean_text', 'num_chars',
       'num_words', 'avg_word_length', 'flesch_score', 'subjectivity_score',
       'neg', 'neu', 'pos', 'compound', 'avg_tfidf ', 'Max_tfidf ',
       'avg_tfidf_feature', 'max_tfidf_feature', 'assertive_verbs_count',
       'factive_verbs_count', 'hedges_count', 'implicative_verbs_count',
       'report_verbs_count', 'bias_word_list_01_2018_count',
       'subjectivityClues_lexicon_count', 'pos_tags', 'lemma_pos', 'Ner_tags',
       'NNS_count', 'NNPS_count', 'DT_count', 'JJ_count', 'JJS_count',
       'NN_count', 'NORP_count', 'PERSON_count', 'adverb_count', 'GPE_count',
       'pos_tags', 'lemmatized_withStopwords',
       'tokenized_lemmatized_withstopwords', 'lemma_pos', 'Ner_tags'],
      dtype='object')

In [ ]:
import ast

for word, tag in ast.literal_eval(scoring_features.pos_tags[0]):
  print(word , "->", tag)

many -> JJ
people -> NNS
live -> VBP
in -> IN
ethiopia -> NNP
the -> DT
people -> NNS
be -> VB
very -> RB
thin -> JJ
and -> CC
good -> JJ
at -> IN
distance -> NN
run -> NN


In [ ]:
import ast

def pos_count(text):
  pos_type = []
  # Combining lists of lists into single list 
  pos_list = ast.literal_eval(text)
  for word,tag in pos_list :
    if tag == part_of_speech:
      # pos_type.append(word)
      return 1
  return 0

In [ ]:
def check_col(col_name):
  length = len(scoring_features[scoring_features[col_name] != 0])
  return length 

In [ ]:
def drop_col(df,col_name):
  df.drop([col_name],axis=1, inplace=True)
  print(df.columns)

In [ ]:
part_of_speech = 'NNS' # Plural nouns
scoring_features['NNS_count'] = scoring_features['pos_tags'].apply(pos_count)

In [ ]:
check_col('NNS_count')

2025

In [ ]:
part_of_speech = 'NNPS' # Proper Plural nouns
scoring_features['NNPS_count'] = scoring_features['pos_tags'].apply(pos_count)

In [ ]:
check_col('NNPS_count')

847

In [ ]:
part_of_speech = 'DT' # Determiners ( The with adjectives to refer a whole group of people)
scoring_features['DT_count'] = scoring_features['pos_tags'].apply(pos_count)

In [ ]:
check_col('DT_count')

12602

In [ ]:
part_of_speech = 'JJ' # Adjective
scoring_features['JJ_count'] = scoring_features['pos_tags'].apply(pos_count)

In [ ]:
check_col('JJ_count')

12399

In [ ]:
part_of_speech = 'sb' # Subject ( Subject refering to the group)
scoring_features['sb_count'] = scoring_features['pos_tags'].apply(pos_count)

In [ ]:
check_col('sb_count')

0

In [ ]:
drop_col(scoring_features,'sb_count')

Index(['Ethnicity', 'gender', 'profession', 'religion', 'Anti-stereotype',
       'stereotype', 'unrelated', 'sentence', 'clean_text',
       'lemmatized_withStopwords', 'tokenized_lemmatized_withstopwords',
       'num_chars', 'num_words', 'avg_word_length', 'flesch_score',
       'subjectivity_score', 'neg', 'neu', 'pos', 'compound', 'avg_tfidf ',
       'Max_tfidf ', 'avg_tfidf_feature', 'max_tfidf_feature',
       'assertive_verbs_count', 'factive_verbs_count', 'hedges_count',
       'implicative_verbs_count', 'report_verbs_count',
       'bias_word_list_01_2018_count', 'subjectivityClues_lexicon_count',
       'pos_tags', 'lemmatized_withStopwords',
       'tokenized_lemmatized_withstopwords', 'lemma_pos', 'Ner_tags',
       'NNS_count', 'NNPS_count', 'DT_count', 'JJ_count', 'JJS_count',
       'NN_count', 'NORP_count', 'PERSON_count'],
      dtype='object')


In [ ]:
part_of_speech = 'JJS' # Superlative adjective for subjectivity
scoring_features['JJS_count'] = scoring_features['pos_tags'].apply(pos_count)

In [ ]:
check_col('JJS_count')

279

In [ ]:
part_of_speech = 'JJ' # adjective indicates 
scoring_features['JJ_count'] = scoring_features['pos_tags'].apply(pos_count)

In [ ]:
check_col('JJ_count')

12399

In [ ]:
part_of_speech = 'NN' # Noun
scoring_features['NN_count'] = scoring_features['pos_tags'].apply(pos_count)

In [ ]:
check_col('NN_count')

15371

In [ ]:
part_of_speech = "RB" # Frequency adverb to indicate generic sentences ['usually', 'typically', 'generally', 'sometimes', 'always']
scoring_features['adverb_count'] = scoring_features['pos_tags'].apply(pos_count)

In [ ]:
check_col('adverb_count')

7636

In [ ]:
scoring_features.columns

Index(['Ethnicity', 'gender', 'profession', 'religion', 'Anti-stereotype',
       'stereotype', 'unrelated', 'sentence', 'clean_text',
       'lemmatized_withStopwords', 'tokenized_lemmatized_withstopwords',
       'num_chars', 'num_words', 'avg_word_length', 'flesch_score',
       'subjectivity_score', 'neg', 'neu', 'pos', 'compound', 'avg_tfidf ',
       'Max_tfidf ', 'avg_tfidf_feature', 'max_tfidf_feature',
       'assertive_verbs_count', 'factive_verbs_count', 'hedges_count',
       'implicative_verbs_count', 'report_verbs_count',
       'bias_word_list_01_2018_count', 'subjectivityClues_lexicon_count',
       'pos_tags', 'lemmatized_withStopwords',
       'tokenized_lemmatized_withstopwords', 'lemma_pos', 'Ner_tags',
       'NNS_count', 'NNPS_count', 'DT_count', 'JJ_count', 'JJS_count',
       'NN_count', 'NORP_count', 'PERSON_count', 'verb_count', 'article_count',
       'adverb_count'],
      dtype='object')

Named entity recognition features


In [ ]:
part_of_speech = 'NORP' # Nationalities or religious or political groups
scoring_features['NORP_count'] = scoring_features['Ner_tags'].apply(pos_count)

In [ ]:
check_col('NORP_count')

3282

In [ ]:
part_of_speech = 'PERSON' # People, including fictional => cue for gender, 
scoring_features['PERSON_count'] = scoring_features['Ner_tags'].apply(pos_count)

In [ ]:
check_col('PERSON_count')

1551

In [ ]:
part_of_speech = 'GPE' # Countires, cities, states
scoring_features['GPE_count'] = scoring_features['Ner_tags'].apply(pos_count)

In [ ]:
check_col('GPE_count')

2239

Characteristic terms used in stereoset and crows-s-pair dataset per bias type

In [ ]:
pip install scattertext

     |████████████████████████████████| 7.3 MB 4.7 MB/s 
     |████████████████████████████████| 23.9 MB 95 kB/s 
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9310 sha256=b39ef2480e29d783e1eb8c96925b29e0712eb47b15f9c2af04ae5b94fac1abc5
  Stored in directory: /root/.cache/pip/wheels/cb/19/58/4e8fdd0009a7f89dbce3c18fff2e0d0fa201d5cdfd16f113b7
Successfully built flashtext
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import scattertext as st
import spacy
from pprint import pprint
import pandas as pd

In [ ]:
stereo = pd.read_csv('/content/drive/MyDrive/Trained_models/mult_label_dataset/Unrelated_samples_adjusted/Multi_label_stereo.csv',index_col = 0)

In [ ]:
stereo.bias_type.value_counts()

Ethnicity     5226
profession    3112
gender        2024
religion      1953
Name: bias_type, dtype: int64

In [ ]:
corpus = st.CorpusFromPandas(stereo, category_col='bias_type', text_col='sentence', nlp=nlp).build()

In [ ]:
x = pd.DataFrame(corpus.get_scaled_f_scores_vs_background())

In [ ]:
x

,corpus,background,Scaled f-score
eriteria,96.0,0.0,0.001173
norweigan,96.0,46910.0,0.000911
eritrean,101.0,229521.0,0.000514
allcaps,41.0,0.0,0.000501
crimean,100.0,279156.0,0.000452
...,...,...,...
genlyte,0.0,16620.0,0.000000
genlock,0.0,32902.0,0.000000
genl,0.0,121289.0,0.000000
genksyms,0.0,32575.0,0.000000


Extracting top 1000 keyterms for each bias type

In [ ]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df['Ethnicity_score'] = corpus.get_scaled_f_scores('Ethnicity')
pprint(list(term_freq_df.sort_values(by='Ethnicity_score', ascending=False).index[:20]))

['ethiopia',
 'italy',
 'somalia',
 'sierra',
 'lebanon',
 'japanese',
 'persian',
 'bangladesh',
 'ghanaian',
 'morocco',
 'ecuador',
 'spain',
 'cameroon',
 'leon',
 'sierra leon',
 'eritrean',
 'persian people',
 'crimean',
 'bengali',
 'norweigan']


In [ ]:
charteristic_terms_ethnicity = list(term_freq_df.sort_values(by='Ethnicity_score', ascending=False).index[:1000])

In [ ]:
term_freq_df['profession_score'] = corpus.get_scaled_f_scores('profession')
charteristic_terms_profession = list(term_freq_df.sort_values(by='profession_score', ascending=False).index[:1000])

In [ ]:
term_freq_df['gender_score'] = corpus.get_scaled_f_scores('gender')
charteristic_terms_gender = list(term_freq_df.sort_values(by='gender_score', ascending=False).index[:1000])

In [ ]:
term_freq_df['religion_score'] = corpus.get_scaled_f_scores('religion')
charteristic_terms_religion = list(term_freq_df.sort_values(by='religion_score', ascending=False).index[:1000])

In [ ]:
def count_lexicon(text):
  count = 0
  try:
    for token in lexicon:
      if token in text and len(token) > 1:
        count +=1
      else:
        continue
  except :
    pass
  return count

Charteristic_terms Ethnicity 

In [ ]:
lexicon = charteristic_terms_ethnicity 

In [ ]:
scoring_features['charteristic_terms_ethnicity_count'] = scoring_features['tokenized_lemmatized_withstopwords'].apply(count_lexicon)

In [ ]:
len(scoring_features[scoring_features['charteristic_terms_ethnicity_count'] != 0])

16019

Charteristic_terms_profession 

In [ ]:
lexicon = charteristic_terms_profession

In [ ]:
lexicon[:10]

['schoolboy',
 'gentlemen',
 'schoolgirl',
 'the schoolboy',
 'mommy',
 'the gentlemen',
 'the schoolgirl',
 'herself',
 'grandfather',
 'my grandfather']

In [ ]:
scoring_features['charteristic_terms_profession_count'] = scoring_features['tokenized_lemmatized_withstopwords'].apply(count_lexicon)

In [ ]:
len(scoring_features[scoring_features['charteristic_terms_profession_count'] != 0])

15650

Charteristic_terms_gender

In [ ]:
lexicon = set(tokenize(lexicons['assertive_verbs.txt']))

In [ ]:
lexicon[:10]

['schoolboy',
 'gentlemen',
 'schoolgirl',
 'the schoolboy',
 'mommy',
 'the gentlemen',
 'the schoolgirl',
 'herself',
 'grandfather',
 'my grandfather']

In [ ]:
scoring_features['charteristic_terms_gender_count'] = scoring_features['tokenized_lemmatized_withstopwords'].apply(count_lexicon)

In [ ]:
len(scoring_features[scoring_features['charteristic_terms_gender_count'] != 0])

14758

Charteristic_terms_religion

In [ ]:
lexicon = charteristic_terms_religion

In [ ]:
lexicon[:10]

['difference between',
 's the',
 'what s',
 'between a',
 'you call',
 'it s',
 'don t',
 'don',
 'the jews',
 'jesus']

In [ ]:
scoring_features['charteristic_terms_religion_count'] = scoring_features['tokenized_lemmatized_withstopwords'].apply(count_lexicon)

In [ ]:
len(scoring_features[scoring_features['charteristic_terms_religion_count'] != 0])

15586

## Training

### SVM with selected features

In [ ]:
MAX_LEN = 50
RANDOM_SEED = 42

In [ ]:
import pandas as pd
feature_df = pd.read_csv('/content/drive/MyDrive/Trained_models/Results_compilation /Unrelated_adjusted/Baselines/Feature based/Feature_df/final_features.csv', index_col = 0)

In [ ]:
feature_df.head()

,Ethnicity,gender,profession,religion,Anti-stereotype,stereotype,unrelated,num_chars,num_words,avg_word_length,flesch_score,neg,neu,pos,max_tfidf_feature,assertive_verbs_count,factive_verbs_count,hedges_count,implicative_verbs_count,report_verbs_count,bias_word_list_01_2018_count,subjectivityClues_lexicon_count,NNS_count,NNPS_count,DT_count,JJ_count,JJS_count,NN_count,NORP_count,PERSON_count,adverb_count,GPE_count,charteristic_terms_ethnicity_count,charteristic_terms_profession_count,charteristic_terms_gender_count,charteristic_terms_religion_count
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,84,15,4.666667,89.24,0.000,0.816,0.184,0.442653,0,0,1,0,0,4,5,1,0,1,1,0,1,0,0,1,1,10,3,2,2
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,53,10,4.400000,52.87,0.000,1.000,0.000,0.494477,0,0,0,0,0,2,12,0,0,1,1,0,1,1,0,1,0,11,6,3,5
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,79,14,4.714286,89.75,0.000,1.000,0.000,0.510763,0,0,1,1,0,2,3,0,0,1,0,0,1,0,0,0,0,4,3,9,9
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,106,20,4.350000,86.71,0.105,0.759,0.135,0.344728,0,0,2,0,0,4,9,1,0,1,1,0,1,0,0,1,0,14,3,3,5
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,58,11,4.363636,91.27,0.000,0.690,0.310,0.508637,0,1,0,0,0,2,3,1,0,0,1,0,1,1,0,0,0,8,2,0,4


In [ ]:
# feature_df.dropna(axis=1,inplace=True)

In [ ]:
feature_df.iloc[:,7:].columns

Index(['num_chars', 'num_words', 'avg_word_length', 'flesch_score', 'neg',
       'neu', 'pos', 'max_tfidf_feature', 'assertive_verbs_count',
       'factive_verbs_count', 'hedges_count', 'implicative_verbs_count',
       'report_verbs_count', 'bias_word_list_01_2018_count',
       'subjectivityClues_lexicon_count', 'NNS_count', 'NNPS_count',
       'DT_count', 'JJ_count', 'JJS_count', 'NN_count', 'NORP_count',
       'PERSON_count', 'adverb_count', 'GPE_count',
       'charteristic_terms_ethnicity_count',
       'charteristic_terms_profession_count',
       'charteristic_terms_gender_count', 'charteristic_terms_religion_count'],
      dtype='object')

In [ ]:
y = feature_df.iloc[:,:7].values
X = feature_df.iloc[:,7:].values

In [ ]:
LABEL_COLUMN = ['Ethnicity',	'gender'	,'profession'	,'religion',	'Anti-stereotype',	'stereotype',	'unrelated']

In [ ]:
features = feature_df.iloc[:,7:]
FEATURE_COLUMNS = features.columns

In [ ]:
features.columns

Index(['num_chars', 'num_words', 'avg_word_length', 'flesch_score', 'neg',
       'neu', 'pos', 'max_tfidf_feature', 'assertive_verbs_count',
       'factive_verbs_count', 'hedges_count', 'implicative_verbs_count',
       'report_verbs_count', 'bias_word_list_01_2018_count',
       'subjectivityClues_lexicon_count', 'NNS_count', 'NNPS_count',
       'DT_count', 'JJ_count', 'JJS_count', 'NN_count', 'NORP_count',
       'PERSON_count', 'adverb_count', 'GPE_count',
       'charteristic_terms_ethnicity_count',
       'charteristic_terms_profession_count',
       'charteristic_terms_gender_count', 'charteristic_terms_religion_count'],
      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split

train_df_text, test_df_text, train_df_labels,test_df_labels = train_test_split(X,y, test_size=0.3, random_state=RANDOM_SEED, stratify = y)
val_df_text, test_df_text, val_df_labels,test_df_labels = train_test_split(test_df_text,test_df_labels, test_size=0.5, random_state=RANDOM_SEED,stratify = test_df_labels)

In [ ]:
train_df_labels = pd.DataFrame(train_df_labels, columns= LABEL_COLUMN)
val_df_labels = pd.DataFrame(val_df_labels, columns= LABEL_COLUMN)
test_df_labels = pd.DataFrame(test_df_labels, columns= LABEL_COLUMN)
train_df_features = pd.DataFrame(train_df_text, columns = FEATURE_COLUMNS)
val_df_features  = pd.DataFrame(val_df_text, columns = FEATURE_COLUMNS)
test_df_features  = pd.DataFrame(test_df_text, columns = FEATURE_COLUMNS)

In [ ]:
# train_df = pd.concat([train_df_text,train_df_labels], axis = 1)
# val_df = pd.concat([val_df_text,val_df_labels], axis = 1)
# test_df = pd.concat([test_df_text,test_df_labels], axis = 1)

Metrics

In [ ]:
def Accuracy(y_true, y_pred):
  temp = 0
  for i in range(y_true.shape[0]):
      temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
  return temp / y_true.shape[0]

In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report,hamming_loss, roc_auc_score, accuracy_score,multilabel_confusion_matrix, precision_recall_fscore_support
import numpy as np
import json

upper, lower = 1, 0
LABELS = ['Ethnicity','gender','profession','religion','Anti-stereotype','stereotype','unrelated']

def classification_metrics(test_pred,labels,model_name,threshold, sigmoid = False):

  print("Evaluation metrics for test set:")
  if sigmoid:
    y_pred = np.where(test_pred > threshold, upper, lower)
  else:
    y_pred = test_pred

  ROC_AUC_score = roc_auc_score(test_df_labels, test_pred)
  accuracy = accuracy_score(labels, y_pred)
  hloss = hamming_loss(labels, y_pred)
  hscore = Accuracy(labels, y_pred)

  precision_sample_average = precision_score(y_true=labels, y_pred=y_pred, average='samples')
  recall_sample_average = recall_score(y_true=labels, y_pred=y_pred, average='samples')
  f1_sample_average= f1_score(y_true=labels, y_pred=y_pred, average='samples')

  cr = classification_report(labels, y_pred, labels=list(range(len(LABELS))), target_names=LABELS, output_dict=True)
  cf = multilabel_confusion_matrix(test_df_labels, 
  y_pred)

  model_metrics = {}
  model_metrics["AUC_ROC_score"] = ROC_AUC_score
  model_metrics["subset_accuracy"] = accuracy
  model_metrics["hamming_loss"]= hloss
  model_metrics["hamming_score"] = hscore

  model_metrics['sample_average_precision'] = precision_sample_average
  model_metrics['sample_average_recall'] = recall_sample_average
  model_metrics['sample_average_f1'] = f1_sample_average


  if write_to_file:
    model_metrics["Classification_report"] = cr

    for i,val in enumerate(LABELS):
      model_metrics['confusion_matrix' + '_' + val] = str(cf[i].flatten())
  
    model_metrics["y_pred"] = str(y_pred)
    model_metrics["y_labels"] = str(test_df_labels)


    if threshold != 0.5:
      th = "calculated_threshold"
    else:
      th = threshold

    model_metrics["threshold"] = th
    output_file = "eval_results_" + model_name + "_"+str(th) +"_"+ ".json"
    output_file = '/content/drive/MyDrive/Trained_models/Results_compilation /Unrelated_adjusted/Baselines/Feature based'+output_file
    with open(output_file, "w" ) as writer:
        json.dump(model_metrics,writer)
  

  print("\n ROC-AUC score: %.6f \n" % (ROC_AUC_score))
  print("\n Subset accuracy : %.6f \n" % (accuracy))
  print("\n hamming_loss : %.6f \n" % (hloss))
  print("\n hamming score : %.6f \n" % hscore)
  print("\n sample average  precision_sample_average : %.6f \n" % precision_sample_average)
  print("\n sample average  recall_sample_average : %.6f \n" % recall_sample_average)
  print("\n sample average  f1_sample_average : %.6f \n" % f1_sample_average)
  

  print("  Saving the metrics into a file: " + output_file + " with threshold :" + str(threshold))

Without feature scaling

In [ ]:
train_df_features.shape, train_df_labels.shape

((11592, 29), (11592, 7))

In [ ]:
val_df_features.shape, val_df_labels.shape

((2484, 29), (2484, 7))

In [ ]:
np.any(np.isnan(train_df_features))

False

In [ ]:
train_df_features = pd.concat([train_df_features,val_df_features])
train_df_labels = pd.concat([train_df_labels,val_df_labels])

In [ ]:
train_df_features.reset_index(drop=True,inplace=True)
train_df_labels.reset_index(drop=True,inplace=True)

In [ ]:
np.any(np.isnan(train_df_features))

False

In [ ]:
train_df_features

,num_chars,num_words,avg_word_length,flesch_score,neg,neu,pos,max_tfidf_feature,assertive_verbs_count,factive_verbs_count,hedges_count,implicative_verbs_count,report_verbs_count,bias_word_list_01_2018_count,subjectivityClues_lexicon_count,NNS_count,NNPS_count,DT_count,JJ_count,JJS_count,NN_count,NORP_count,PERSON_count,adverb_count,GPE_count,charteristic_terms_ethnicity_count,charteristic_terms_profession_count,charteristic_terms_gender_count,charteristic_terms_religion_count
0,104.0,19.0,4.526316,78.75,0.000,0.865,0.135,0.451396,0.0,0.0,2.0,0.0,0.0,3.0,7.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,8.0,5.0,4.0,5.0
1,56.0,12.0,3.750000,93.14,0.161,0.839,0.000,0.527550,1.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,7.0,2.0,3.0,2.0
2,96.0,17.0,4.705882,71.31,0.191,0.698,0.112,0.753407,0.0,1.0,2.0,0.0,1.0,6.0,8.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,9.0,8.0,5.0
3,76.0,13.0,4.923077,73.34,0.136,0.679,0.185,0.476005,0.0,1.0,1.0,0.0,0.0,3.0,4.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,3.0,4.0,5.0
4,52.0,9.0,4.777778,53.88,0.000,0.727,0.273,0.510271,0.0,0.0,2.0,0.0,0.0,1.0,5.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,5.0,6.0,4.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14071,74.0,13.0,4.769231,83.66,0.155,0.845,0.000,0.428266,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,4.0
14072,32.0,7.0,3.714286,115.13,0.000,0.714,0.286,0.654300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,3.0,4.0,2.0
14073,69.0,13.0,4.384615,90.26,0.000,0.647,0.353,0.447422,0.0,1.0,0.0,1.0,0.0,2.0,3.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,6.0,1.0,3.0,4.0
14074,28.0,5.0,4.800000,100.24,0.000,1.000,0.000,0.512425,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,3.0,2.0


In [ ]:
from sklearn.svm import SVC # To be run again after running the above cell (train, test : 85, 15)
from sklearn.multioutput import MultiOutputClassifier

classifier = SVC(kernel = 'linear', random_state = 42)
multilabel_classifier = MultiOutputClassifier(classifier, n_jobs=-1)
multilabel_classifier = multilabel_classifier.fit(train_df_features, train_df_labels)

Prediction on test set

In [ ]:
y_test_pred = multilabel_classifier.predict(test_df_features)

In [ ]:
y_test_pred

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
labels = test_df_labels.values

In [ ]:
write_to_file = True
classification_metrics(y_test_pred,labels,"SVM_Only_features",0.5)

Evaluation metrics for test set:

 ROC-AUC score: 0.692699 


 Subset accuracy : 0.274155 


 hamming_loss : 0.175063 


 hamming score : 0.437970 


 sample average  precision_sample_average : 0.578167 


 sample average  recall_sample_average : 0.458736 


 sample average  f1_sample_average : 0.496578 

  Saving the metrics into a file: eval_results_SVM_Only_features_0.5_.json with threshold :0.5


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM with selected features and tfidf features

SVM + tfi_idf + feature scaling

In [ ]:
feature_vector_tfidf = pd.concat([feature_df,tf_idf_feature], axis = 1)

In [ ]:
feature_vector_tfidf

,Ethnicity,gender,profession,religion,Anti-stereotype,stereotype,unrelated,num_chars,num_words,avg_word_length,flesch_score,neg,neu,pos,max_tfidf_feature,assertive_verbs_count,factive_verbs_count,hedges_count,implicative_verbs_count,report_verbs_count,bias_word_list_01_2018_count,subjectivityClues_lexicon_count,NNS_count,NNPS_count,DT_count,JJ_count,JJS_count,NN_count,NORP_count,PERSON_count,adverb_count,GPE_count,charteristic_terms_ethnicity_count,charteristic_terms_profession_count,charteristic_terms_gender_count,charteristic_terms_religion_count,tfIdf_aardvark,tfIdf_ab,tfIdf_ababa,tfIdf_aback,...,tfIdf_yiddishkeit,tfIdf_yield,tfIdf_yo,tfIdf_yoga,tfIdf_yogurt,tfIdf_yolanda,tfIdf_york,tfIdf_yorker,tfIdf_yorkshire,tfIdf_young,tfIdf_youth,tfIdf_youtube,tfIdf_yrs,tfIdf_yu,tfIdf_yucatan,tfIdf_yum,tfIdf_yummy,tfIdf_zach,tfIdf_zack,tfIdf_zag,tfIdf_zaknelson,tfIdf_ze,tfIdf_zebra,tfIdf_zeke,tfIdf_zenlike,tfIdf_zero,tfIdf_zig,tfIdf_zionism,tfIdf_zionist,tfIdf_zip,tfIdf_zit,tfIdf_zoey,tfIdf_zog,tfIdf_zombie,tfIdf_zone,tfIdf_zoo,tfIdf_zookeeper,tfIdf_zoos,tfIdf_zumba,tfIdf_zyklon
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,84,15,4.666667,89.24,0.000,0.816,0.184,0.442653,0,0,1,0,0,4,5,1,0,1,1,0,1,0,0,1,1,10,3,2,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,53,10,4.400000,52.87,0.000,1.000,0.000,0.494477,0,0,0,0,0,2,12,0,0,1,1,0,1,1,0,1,0,11,6,3,5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,79,14,4.714286,89.75,0.000,1.000,0.000,0.510763,0,0,1,1,0,2,3,0,0,1,0,0,1,0,0,0,0,4,3,9,9,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,106,20,4.350000,86.71,0.105,0.759,0.135,0.344728,0,0,2,0,0,4,9,1,0,1,1,0,1,0,0,1,0,14,3,3,5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,58,11,4.363636,91.27,0.000,0.690,0.310,0.508637,0,1,0,0,0,2,3,1,0,0,1,0,1,1,0,0,0,8,2,0,4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16555,0.0,0.0,0.0,0.0,0.0,0.0,1.0,46,7,5.714286,64.37,0.000,0.674,0.326,0.705372,0,0,0,0,0,2,3,0,0,0,1,0,1,0,0,0,0,4,1,3,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16556,0.0,0.0,0.0,0.0,0.0,0.0,1.0,54,9,5.111111,29.52,0.000,0.715,0.285,0.447653,0,0,1,0,0,2,6,0,0,0,1,0,1,0,1,1,0,5,3,2,5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16557,0.0,0.0,0.0,0.0,0.0,0.0,1.0,73,9,7.222222,3.12,0.000,1.000,0.000,0.416111,0,0,1,0,0,2,5,0,0,1,1,0,1,0,0,0,0,3,5,4,4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16558,0.0,0.0,0.0,0.0,0.0,0.0,1.0,141,31,3.580645,82.31,0.000,1.000,0.000,0.410444,0,1,2,0,1,2,4,0,0,1,1,0,1,0,0,1,0,7,4,6,10,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [ ]:
y = feature_vector_tfidf.iloc[:,:7].values
X = feature_vector_tfidf.iloc[:,7:].values

In [ ]:
X

array([[ 84.        ,  15.        ,   4.66666667, ...,   0.        ,
          0.        ,   0.        ],
       [ 53.        ,  10.        ,   4.4       , ...,   0.        ,
          0.        ,   0.        ],
       [ 79.        ,  14.        ,   4.71428571, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [ 73.        ,   9.        ,   7.22222222, ...,   0.        ,
          0.        ,   0.        ],
       [141.        ,  31.        ,   3.58064516, ...,   0.        ,
          0.        ,   0.        ],
       [ 65.        ,  13.        ,   4.        , ...,   0.        ,
          0.        ,   0.        ]])

In [ ]:
y

array([[1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
LABEL_COLUMN = ['Ethnicity',	'gender'	,'profession'	,'religion',	'Anti-stereotype',	'stereotype',	'unrelated']

In [ ]:
features = feature_vector_tfidf.iloc[:,7:]
FEATURE_COLUMNS = features.columns

In [ ]:
features.columns

Index(['num_chars', 'num_words', 'avg_word_length', 'flesch_score', 'neg',
       'neu', 'pos', 'max_tfidf_feature', 'assertive_verbs_count',
       'factive_verbs_count',
       ...
       'tfIdf_zit', 'tfIdf_zoey', 'tfIdf_zog', 'tfIdf_zombie', 'tfIdf_zone',
       'tfIdf_zoo', 'tfIdf_zookeeper', 'tfIdf_zoos', 'tfIdf_zumba',
       'tfIdf_zyklon'],
      dtype='object', length=9058)

In [ ]:
from sklearn.model_selection import train_test_split

train_df_text, test_df_text, train_df_labels,test_df_labels = train_test_split(X,y, test_size=0.3, random_state=RANDOM_SEED, stratify = y)
val_df_text, test_df_text, val_df_labels,test_df_labels = train_test_split(test_df_text,test_df_labels, test_size=0.5, random_state=RANDOM_SEED,stratify = test_df_labels)

In [ ]:
train_df_labels = pd.DataFrame(train_df_labels, columns= LABEL_COLUMN)
val_df_labels = pd.DataFrame(val_df_labels, columns= LABEL_COLUMN)
test_df_labels = pd.DataFrame(test_df_labels, columns= LABEL_COLUMN)
train_df_features = pd.DataFrame(train_df_text, columns = FEATURE_COLUMNS)
val_df_features  = pd.DataFrame(val_df_text, columns = FEATURE_COLUMNS)
test_df_features  = pd.DataFrame(test_df_text, columns = FEATURE_COLUMNS)

In [ ]:
train_df_features = pd.concat([train_df_features,val_df_features])
train_df_labels = pd.concat([train_df_labels,val_df_labels])

In [ ]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(train_df_features)
# X_test = sc.transform(test_df_features)

In [ ]:
X_train

NameError: ignored

In [ ]:
from sklearn.svm import SVC # To be run again after running the above cell (train, test : 85, 15)
from sklearn.multioutput import MultiOutputClassifier

classifier = SVC(kernel = 'linear', random_state = 42)
multilabel_classifier = MultiOutputClassifier(classifier, n_jobs=-1)
multilabel_classifier = multilabel_classifier.fit(train_df_features, train_df_labels)

KeyboardInterrupt: ignored

In [ ]:
y_test_pred = multilabel_classifier.predict(X_test)

In [ ]:
X_test.shape

In [ ]:
labels = test_df_labels.values
labels.shape

In [ ]:
write_to_file = True
classification_metrics(y_test_pred,labels,"SVM_tfidf_Selectedfeatures",0.5)

### Naive Bayes with bag of words features

Naive Bayes with bag of words features

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Trained_models/Results_compilation /Unrelated_adjusted/Baselines/Feature based/Feature_df/stereo_features_pos_ner.csv',index_col = 0)

In [ ]:
df

,Ethnicity,gender,profession,religion,Anti-stereotype,stereotype,unrelated,sentence,clean_text,pos_tags,lemmatized_withStopwords,tokenized_lemmatized_withstopwords,lemma_pos,Ner_tags
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Many people live in Ethiopia. The people are v...,many people live in ethiopia the people be ver...,"[('many', 'JJ'), ('people', 'NNS'), ('live', '...",people live ethiopia people thin good distance...,"['people', 'live', 'ethiopia', 'people', 'thin...","[('people', 'NNS'), ('live', 'VBP'), ('ethiopi...","[('ethiopia', 'GPE')]"
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,My professor is a hispanic man He came here il...,professor be a hispanic man come here illegally,"[('professor', 'NNP'), ('be', 'VB'), ('a', 'DT...",professor hispanic man come illegally,"['professor', 'hispanic', 'man', 'come', 'ille...","[('professor', 'NNP'), ('hispanic', 'NNP'), ('...","[('hispanic', 'NORP')]"
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,The schoolgirl is walking down the street. She...,the schoolgirl be walk down the street be a ta...,"[('the', 'DT'), ('schoolgirl', 'NN'), ('be', '...",schoolgirl walk street target child predator,"['schoolgirl', 'walk', 'street', 'target', 'ch...","[('schoolgirl', 'NNP'), ('walk', 'NNP'), ('str...",[]
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,We'll likely fly over Cape Verde on our flight...,will likely fly over cape verde on flight tomo...,"[('will', 'MD'), ('likely', 'RB'), ('fly', 'VB...",likely fly cape verde flight tomorrow bet peop...,"['likely', 'fly', 'cape', 'verde', 'flight', '...","[('likely', 'RB'), ('fly', 'VB'), ('cape', 'NN...","[('tomorrow', 'DATE')]"
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,I know many people who are Russian. They love ...,know many people who be russian love vodka,"[('know', 'VBP'), ('many', 'JJ'), ('people', '...",know people russian love vodka,"['know', 'people', 'russian', 'love', 'vodka']","[('know', 'VBP'), ('people', 'NNS'), ('russian...","[('russian', 'NORP')]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16555,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Cookies are good substitutes with liquid milk.,cookie be good substitute with liquid milk,"[('cookie', 'NN'), ('be', 'VB'), ('good', 'JJ'...",cookie good substitute liquid milk,"['cookie', 'good', 'substitute', 'liquid', 'mi...","[('cookie', 'NNP'), ('good', 'JJ'), ('substitu...",[]
16556,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Jollof - Rice cereal is totally better than Ja...,jollof rice cereal be totally well than jambalaya,"[('jollof', 'NNP'), ('rice', 'NNP'), ('cereal'...",jollof rice cereal totally jambalaya,"['jollof', 'rice', 'cereal', 'totally', 'jamba...","[('jollof', 'NNP'), ('rice', 'NNP'), ('cereal'...","[('jollof rice', 'PERSON')]"
16557,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Biking out is an economical energy-efficient m...,bike out be an economical energy efficient mod...,"[('bike', 'NN'), ('out', 'RP'), ('be', 'VB'), ...",bike economical energy efficient mode unk tran...,"['bike', 'economical', 'energy', 'efficient', ...","[('bike', 'NNP'), ('economical', 'JJ'), ('ener...",[]
16558,0.0,0.0,0.0,0.0,0.0,0.0,1.0,I might see the little dog is a still in need ...,may see the little dog be a still in need of f...,"[('may', 'MD'), ('see', 'VB'), ('the', 'DT'), ...",little dog need food park grocery store find f...,"['little', 'dog', 'need', 'food', 'park', 'gro...","[('little', 'JJ'), ('dog', 'NN'), ('need', 'VB...",[]


In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0,len(df)):
  review = re.sub( '[^a-zA-Z]',' ', df['sentence'][i]) # Remove punctuation marks 
  review = review.lower()
  review = review.split()
  # ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not') # Remove the 'not' from stopwords
  # review = [ ps.stem(word) for word in review if not word in set(all_stopwords)] # stem each word if not in stopwords
  review = ' '.join(review) # join with space
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
corpus = df['tokenized_lemmatized_withstopwords'].values

In [ ]:
print(corpus)

["['people', 'live', 'ethiopia', 'people', 'thin', 'good', 'distance', 'run']"
 "['professor', 'hispanic', 'man', 'come', 'illegally']"
 "['schoolgirl', 'walk', 'street', 'target', 'child', 'predator']" ...
 "['bike', 'economical', 'energy', 'efficient', 'mode', 'unk', 'transportation']"
 "['little', 'dog', 'need', 'food', 'park', 'grocery', 'store', 'find', 'food', 'treat']"
 "['california', 'beach', 'morning', 'cool']"]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = None,ngram_range = (1,1), max_df = 1.0) # Further reduce the tokenized words with upper limit
X = cv.fit(corpus)
bow = cv.transform(corpus)

In [ ]:
print("Vocabulary size: {}".format(len(X.vocabulary_)))

Vocabulary size: 9029


In [ ]:
print("Vocabulary content:\n {}".format(X.vocabulary_))

Vocabulary content:
 {'people': 5881, 'live': 4703, 'ethiopia': 2711, 'thin': 8084, 'good': 3403, 'distance': 2323, 'run': 6854, 'professor': 6287, 'hispanic': 3739, 'man': 4873, 'come': 1578, 'illegally': 3926, 'schoolgirl': 6985, 'walk': 8712, 'street': 7725, 'target': 7960, 'child': 1378, 'predator': 6195, 'likely': 4671, 'fly': 3074, 'cape': 1195, 'verde': 8612, 'flight': 3049, 'tomorrow': 8178, 'bet': 781, 'pretty': 6231, 'lazy': 4581, 'know': 4477, 'russian': 6864, 'love': 4775, 'vodka': 8680, 'mark': 4920, 'musician': 5309, 'school': 6983, 'difficulty': 2236, 'drug': 2432, 'complete': 1628, 'high': 3714, 'person': 5909, 'cameroon': 1162, 'dark': 2009, 'moody': 5230, 'ethiopians': 2713, 'sit': 7304, 'huddle': 3855, 'fire': 3003, 'malnourish': 4866, 'savage': 6945, 'afghanistan': 137, 'citizen': 1436, 'middle': 5098, 'east': 2488, 'eastern': 2490, 'terrorist': 8040, 'girl': 3358, 'store': 7699, 'new': 5425, 'york': 8995, 'short': 7213, 'blonde': 856, 'thick': 8081, 'accent': 41, '

In [ ]:
y = df.iloc[:,:7].values

In [ ]:
y

array([[1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
LABEL_COLUMN = ['Ethnicity',	'gender'	,'profession'	,'religion',	'Anti-stereotype',	'stereotype',	'unrelated']

In [ ]:
from sklearn.model_selection import train_test_split

train_df_text, test_df_text, train_df_labels,test_df_labels = train_test_split(bow,y, test_size=0.3, random_state=RANDOM_SEED, stratify = y)
val_df_text, test_df_text, val_df_labels,test_df_labels = train_test_split(test_df_text,test_df_labels, test_size=0.5, random_state=RANDOM_SEED,stratify = test_df_labels)

In [ ]:
train_df_text.shape

(11592, 9029)

In [ ]:
train_df_labels.shape

(11592, 7)

In [ ]:
val_df_text.shape

(2484, 9029)

In [ ]:
val_df_labels.shape

(2484, 7)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier


classifier = MultinomialNB()
multilabel_classifier = MultiOutputClassifier(classifier, n_jobs=-1)
# multilabel_classifier = multilabel_classifier.fit(train_df_text,train_df_labels)
multilabel_classifier = multilabel_classifier.fit(X_train, train_df_labels)

ValueError: ignored

In [ ]:
y_test_pred = multilabel_classifier.predict(test_df_text)

In [ ]:
labels = test_df_labels

In [ ]:
write_to_file = True
classification_metrics(y_test_pred,labels,"MultinomialNB_tfidf",0.5)

Evaluation metrics for test set:

 ROC-AUC score: 0.700864 


 Subset accuracy : 0.322866 


 hamming_loss : 0.154072 


 hamming score : 0.478630 


 sample average  precision_sample_average : 0.620035 


 sample average  recall_sample_average : 0.492754 


 sample average  f1_sample_average : 0.534890 

  Saving the metrics into a file: eval_results_MultinomialNB_tfidf_0.5_.json with threshold :0.5


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### KNN with countVectorizer

In [ ]:
feature_vector_tfidf = pd.concat([df,tf_idf_feature], axis = 1)

In [ ]:
feature_vector_tfidf.head()

,Ethnicity,gender,profession,religion,Anti-stereotype,stereotype,unrelated,sentence,clean_text,pos_tags,lemmatized_withStopwords,tokenized_lemmatized_withstopwords,lemma_pos,Ner_tags,tfIdf_aardvark,tfIdf_ab,tfIdf_ababa,tfIdf_aback,tfIdf_abandon,tfIdf_abattoir,tfIdf_abaya,tfIdf_abdoman,tfIdf_abdul,tfIdf_abel,tfIdf_aberration,tfIdf_abide,tfIdf_ability,tfIdf_able,tfIdf_aboion,tfIdf_abolitionist,tfIdf_abominable,tfIdf_abomination,tfIdf_aboout,tfIdf_abouit,tfIdf_abraham,tfIdf_abrasive,tfIdf_abroad,tfIdf_abruptly,tfIdf_absence,tfIdf_absent,...,tfIdf_yiddishkeit,tfIdf_yield,tfIdf_yo,tfIdf_yoga,tfIdf_yogurt,tfIdf_yolanda,tfIdf_york,tfIdf_yorker,tfIdf_yorkshire,tfIdf_young,tfIdf_youth,tfIdf_youtube,tfIdf_yrs,tfIdf_yu,tfIdf_yucatan,tfIdf_yum,tfIdf_yummy,tfIdf_zach,tfIdf_zack,tfIdf_zag,tfIdf_zaknelson,tfIdf_ze,tfIdf_zebra,tfIdf_zeke,tfIdf_zenlike,tfIdf_zero,tfIdf_zig,tfIdf_zionism,tfIdf_zionist,tfIdf_zip,tfIdf_zit,tfIdf_zoey,tfIdf_zog,tfIdf_zombie,tfIdf_zone,tfIdf_zoo,tfIdf_zookeeper,tfIdf_zoos,tfIdf_zumba,tfIdf_zyklon
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Many people live in Ethiopia. The people are v...,many people live in ethiopia the people be ver...,"[('many', 'JJ'), ('people', 'NNS'), ('live', '...",people live ethiopia people thin good distance...,"['people', 'live', 'ethiopia', 'people', 'thin...","[('people', 'NNS'), ('live', 'VBP'), ('ethiopi...","[('ethiopia', 'GPE')]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,My professor is a hispanic man He came here il...,professor be a hispanic man come here illegally,"[('professor', 'NNP'), ('be', 'VB'), ('a', 'DT...",professor hispanic man come illegally,"['professor', 'hispanic', 'man', 'come', 'ille...","[('professor', 'NNP'), ('hispanic', 'NNP'), ('...","[('hispanic', 'NORP')]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,The schoolgirl is walking down the street. She...,the schoolgirl be walk down the street be a ta...,"[('the', 'DT'), ('schoolgirl', 'NN'), ('be', '...",schoolgirl walk street target child predator,"['schoolgirl', 'walk', 'street', 'target', 'ch...","[('schoolgirl', 'NNP'), ('walk', 'NNP'), ('str...",[],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,We'll likely fly over Cape Verde on our flight...,will likely fly over cape verde on flight tomo...,"[('will', 'MD'), ('likely', 'RB'), ('fly', 'VB...",likely fly cape verde flight tomorrow bet peop...,"['likely', 'fly', 'cape', 'verde', 'flight', '...","[('likely', 'RB'), ('fly', 'VB'), ('cape', 'NN...","[('tomorrow', 'DATE')]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,I know many people who are Russian. They love ...,know many people who be russian love vodka,"[('know', 'VBP'), ('many', 'JJ'), ('people', '...",know people russian love vodka,"['know', 'people', 'russian', 'love', 'vodka']","[('know', 'VBP'), ('people', 'NNS'), ('russian...","[('russian', 'NORP')]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
y = feature_vector_tfidf.iloc[:,:7].values
X = feature_vector_tfidf.iloc[:,15:].values

In [ ]:
y

array([[1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from sklearn.model_selection import train_test_split

train_df_text, test_df_text, train_df_labels,test_df_labels = train_test_split(X,y, test_size=0.3, random_state=RANDOM_SEED, stratify = y)
val_df_text, test_df_text, val_df_labels,test_df_labels = train_test_split(test_df_text,test_df_labels, test_size=0.5, random_state=RANDOM_SEED,stratify = test_df_labels)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(train_df_text,train_df_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
y_test_pred = classifier.predict(test_df_text)

In [ ]:
labels = test_df_labels

In [ ]:
write_to_file = True
classification_metrics(y_test_pred,labels,"KNN_countVectorizer",0.5)

Evaluation metrics for test set:

 ROC-AUC score: 0.579531 


 Subset accuracy : 0.260870 


 hamming_loss : 0.257189 


 hamming score : 0.299852 


 sample average  precision_sample_average : 0.330515 


 sample average  recall_sample_average : 0.308172 


 sample average  f1_sample_average : 0.315620 

  Saving the metrics into a file: eval_results_KNN_countVectorizer_0.5_.json with threshold :0.5


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Decision Tree with CV

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(train_df_text,train_df_labels)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [ ]:
y_test_pred = classifier.predict(test_df_text)

In [ ]:
labels = test_df_labels

In [ ]:
write_to_file = True
classification_metrics(y_test_pred,labels,"DecisionTree_tfidf",0.5)

Evaluation metrics for test set:

 ROC-AUC score: 0.821131 


 Subset accuracy : 0.583736 


 hamming_loss : 0.137911 


 hamming score : 0.691291 


 sample average  precision_sample_average : 0.745169 


 sample average  recall_sample_average : 0.744968 


 sample average  f1_sample_average : 0.745035 

  Saving the metrics into a file: eval_results_DecisionTree_tfidf_0.5_.json with threshold :0.5


### Random_forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(train_df_text,train_df_labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
y_test_pred = classifier.predict(test_df_text)

In [ ]:
labels = test_df_labels

In [ ]:
write_to_file = True
classification_metrics(y_test_pred,labels,"RandomForest_tfidf",0.5)

Evaluation metrics for test set:

 ROC-AUC score: 0.791064 


 Subset accuracy : 0.479871 


 hamming_loss : 0.137624 


 hamming score : 0.627281 


 sample average  precision_sample_average : 0.736514 


 sample average  recall_sample_average : 0.665459 


 sample average  f1_sample_average : 0.689144 

  Saving the metrics into a file: eval_results_RandomForest_tfidf_0.5_.json with threshold :0.5


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
